In [1]:
import os

In [2]:
%pwd

'd:\\End-to-End-Pneumonia-detection-using-MLflow-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\End-to-End-Pneumonia-detection-using-MLflow-DVC'

In [5]:
pip install  dagshub


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import dagshub
dagshub.init(repo_owner='mekharemanan', repo_name='End-to-End-Pneumonia-detection-using-MLflow-DVC', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

Accessing as mekharemanan

Initialized MLflow to track repo "mekharemanan/End-to-End-Pneumonia-detection-using-MLflow-DVC"

Repository mekharemanan/End-to-End-Pneumonia-detection-using-MLflow-DVC initialized!

c:\Users\sneha\anaconda3\envs\pnemo\lib\site-packages\mlflow\utils\requirements_utils.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [7]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from classifier.utils.common import read_yaml, create_directories, save_json

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/chest xray dataset",
            mlflow_uri="https://dagshub.com/mekharemanan/End-to-End-Pneumonia-detection-using-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {
                    "loss": self.score[0],
                    "accuracy": self.score[1],
                    "precision": self.score[2],
                    "recall": self.score[3]
                }

        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                            "loss": self.score[0],
                            "accuracy": self.score[1],
                            "precision": self.score[2],
                            "recall": self.score[3]})

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="DenseNetModel")
            else:
                mlflow.keras.log_model(self.model, "model")

In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-08-04 14:07:19,049: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-04 14:07:19,073: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-04 14:07:19,089: INFO: common: created directory at: artifacts]
Found 187 images belonging to 2 classes.
12/12 [==============================] - 31s 2s/step - loss: 0.2661 - accuracy: 0.8770 - precision: 0.8770 - recall: 0.8770
[2025-08-04 14:07:56,761: INFO: common: json file saved at: scores.json]


2025/08/04 14:07:59 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-08-04 14:08:51,558: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 121). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\sneha\AppData\Local\Temp\tmpf0nxrib8\model\data\model\assets
[2025-08-04 14:09:08,140: INFO: builder_impl: Assets written to: C:\Users\sneha\AppData\Local\Temp\tmpf0nxrib8\model\data\model\assets]


c:\Users\sneha\anaconda3\envs\pnemo\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'DenseNetModel' already exists. Creating a new version of this model...
2025/08/04 14:13:52 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: DenseNetModel, version 2
Created version '2' of model 'DenseNetModel'.
